# 발표 소단원 리스트

9-15, 9-19,9-23, 10-2, 10-6, 10-10, 10-14

# 전체 요약자료

- 9-15 : class definition이 optional arguments를 지원하도록 허용하는 metaclass를 정의하고 싶을 떄
- 9-19 : class의 일부분을 instance가 만들어질 때가 아니라 class가 define 되었을 때 초기화 하고 싶다면 메타클래스를 이용합니다.
- 9-23 : exec() 함수를 이용해 code 조각을 실행시켰는데, 나중에 결과를 사용하려고 보니까 없을 때
- 10-2 : user가 'from module import *' 를 사용할 때, module에서 export 되는 목록을 지정해주고 싶다면
- 10-6 : 이미 loaded된 모듈의 source코드가 바뀌어서 다시 reload 하고싶을 때, imp.reload()를 사용합니다.
- 10-10 : import하고싶은 모듈의 이름이 있는데, string일 때
- 10-14 : virtual environment 만들기!

### 9-15) Defining a Metaclass That Takes Optional Arguments 
- class definition이 optional arguments를 지원하도록 허용하는 metaclass를 정의하고 싶을 떄
- 특히 type creation 과정중에 벌어지는 어떤 processing을 control하고싶을 때

- 파이썬에서 클래스를 정의할 때, metaclass를 지정하는 keyword argument가 있습니다. 예를 들면 Abstract base classes를 metaclass로 갖는 클래스는 아래와 같이 만듭니다.

In [2]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):  
    @abstractmethod 
    def read(self, maxsize=None):  
        pass
    
    @abstractmethod  
    def write(self, data):   
        pass 

- 하지만 custum metaclass를 지정할 때에는 addtional keyword arguemnts가 지원됩니다. 그러니까, 아래와 같이 추가할 수 있습니다.

In [ ]:
class Spam(metaclass=MyMeta, debug=True, synchronize=True): 
    ...

- 위처럼 keyword argument를 추가하려면 메타클래스를 정의할 때 아래와 같이 __ prepare __ (), __ new __ (), __ init __ () 메소드에 keword-only argument를 사용해 아래와 같이 정의해주었어야 합니다.

In [ ]:
class MyMeta(type):
    # Optional    
    @classmethod  
    def __prepare__(cls, name, bases, *, debug=False, synchronize=False):  
        # Custom processing 
         #... 
        return super().__prepare__(name, bases)
    
    # Required  
    def __new__(cls, name, bases, ns, *, debug=False, synchronize=False):  
        # Custom processing   
        #... 
        return super().__new__(cls, name, bases, ns)
    
    # Required   
    def __init__(self, name, bases, ns, *, debug=False, synchronize=False):
        # Custom processing
        # ... 
        super().__init__(name, bases, ns)

- metaclass에 위와 같이 addtional keyword argument를 추가하려면 class creation을 포함한 모든 step을 이해하고 있어야 합니다. 왜냐면 extra arguments가 연관된 모든 method에 전달되기 때문입니다.
- class가 만들어질 때 메소드가 호출되는 순서는 __ prepare __ (), __ new __ (), __ init __ () 순
- 보통 metaclass를 만들 때 대개 __ new __ ()나 __ init __ () 둘 중 하나만 정의하는데, key word argument를 정의하려면 둘 다 정의해야 하고, 반드시 compatible signatures로 주어져야 합니다.

- 메타클래스란? 참고 링크 (https://tech.ssut.me/understanding-python-metaclasses/)
- 매직 메소드 관련 참고 링크 (https://corikachu.github.io/articles/python/python-magic-method)
- __ prepare __(metacls, name, bases, **kwds): 메타 클래스 네임스페이스에 대한 dictionary를 만듭니다. 메타 클래스가 이 속성이 없다면 빈 dict()로 초기화 됩니다. dictionary나 아니면 다른 mapping object를 리턴하는 것!

### 9-19)  Initializing Class Members at Definition Time 
- class의 일부분을 instance가 만들어질 때가 아니라 class가 define 되었을 때 초기화 하고 싶다면 메타클래스를 이용합니다.

- mataclass는 class definition이 있는 지점에서 triggered 되는데, 이 때 additional step을 수행할 수 있습니다.

In [9]:
import operator

class StructTupleMeta(type):    
    def __init__(cls, *args, **kwargs):        
        super().__init__(*args, **kwargs)        
        for n, name in enumerate(cls._fields):
            setattr(cls, name, property(operator.itemgetter(n)))
            
class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields = [] 
    def __new__(cls, *args):
        if len(args) != len(cls._fields):
            raise ValueError('{} arguments required'.format(len(cls._fields)))
        return super().__new__(cls,args) 
    
class Stock(StructTuple): 
    _fields = ['name', 'shares', 'price']
    
class Point(StructTuple):  
    _fields = ['x', 'y'] 

In [13]:
s = Stock('ACME', 50, 91.1)
s

('ACME', 50, 91.1)

In [14]:
s[0]

'ACME'

In [15]:
s.name

'ACME'

In [17]:
s.shares * s.price

4555.0

In [18]:
s.shares = 23

AttributeError: can't set attribute

### 9-23) Executing Code with Local Side Effects
- exec() 함수를 이용해 code 조각을 실행시켰는데, 나중에 결과를 사용하려고 보니까 없을 때

In [30]:
a = 13
exec('c = a + 1')
print(c)

14


- 위와 같이 사용한다. 그런데, 아래처럼 함수에서 부르는 경우 에러가 NameError가 뜬다.

In [31]:
def test():
    a = 13
    exec('d = a + 1')
    print(d) 

In [29]:
test()

NameError: name 'd' is not defined

- 이런 상황을 해결하려면 locals() 함수를 exec() 호출 전에 호출해서 local variables의 dictinary를 얻어주어야 합니다.

In [34]:
def test(): 
    a = 13 
    loc = locals() 
    exec('e = a + 1')
    e = loc['e'] 
    print(b) 

test()

14


### 10-2) Controlling the Import of Everything  
- user가 'from module import *' 를 사용할 때, module에서 export 되는 목록을 지정해주고 싶다면
- __ all __ 이라는 variable을 선언하면 거기에 들어가있는 이름들만 exported 됩니다.

In [35]:
# somemodule.py
def spam():
    pass

def grok():
    pass

blah = 42
# Only export 'spam' and 'grok' 
__all__ = ['spam', 'grok']

- 만약에 __ all __ 을 작성 안하면 이름 앞에 underscore가 없는 모든 것들을 다 export하고, 비어있는 리스트로 작성하면 아무것도 export 안 됩니다

### 10-6) Reloading Modules
- 이미 loaded된 모듈의 source코드가 바뀌어서 다시 reload 하고싶을 때, imp.reload()를 사용합니다.

In [ ]:
>>> import spam 
>>> import imp
>>> imp.reload(spam) 
<module 'spam' from './spam.py'> >>> 

### 10-10)  Importing Modules Using a Name Given in a String 
- import하고싶은 모듈의 이름이 있는데, string일 때
- import command로 불러오고 싶다. 그 때 importlib.import_module() 함수를 사용한다.

In [ ]:
import importlib
>>> math = importlib.import_module('math') 
>>> math.sin(2) 0.9092974268256817 
>>> mod = importlib.import_module('urllib.request') 
>>> u = mod.urlopen('http://www.python.org')

- importlib.import_module() 함수는 import와 똑같은 역할을 하지만, module object를 return해줍니다. 그러면 변수에 넣고 일반적인 모듈을 import 했을 때처럼 사용하시면 됩니다.

### 10-14) Creating a New Python Environment 
- module과 package들을 설치할 수 있는 새로운 python environment를 만들고 싶은데, 새로운 python copy를 설치함으로 인해 시스템에 영향을 주는 일은 피하고 싶을 때
- pyvenv 명령어로 virtual environment를 만들 수 있습니다. 이 command는 python interpreter가 설치된 디렉토리와 같은 디렉토리에 설치되어 있거나, 아니면 Scripts 디렉토리에 있을수도 있습니다.

- virtual environment를 만들면 새로운 디렉토리가 하나 생성됩니다.

bash % pyvenv Spam    
bash %

bash % cd Spam 

bash % ls  
bin               include             lib           pyvenv.cfg 

bash % 

In [ ]:
bash % Spam/bin/python3 

Python 3.3.0 (default, Oct  6 2012, 15:45:22)  
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin 
Type "help", "copyright", "credits" or "license" for more information.
>>> from pprint import pprint   
>>> import sys  
>>> pprint(sys.path)
['',
 '/usr/local/lib/python33.zip',
 '/usr/local/lib/python3.3',
 '/usr/local/lib/python3.3/plat-darwin',
 '/usr/local/lib/python3.3/lib-dynload',
 '/Users/beazley/Spam/lib/python3.3/site-packages'
 ]

- 가상 환경에서는새로 만든 디렉토리의 bin에 설치된 python interpreter를 사용합니다.
- site-packages 디렉토리도 하나 새로 생성되는데, 여기에 third-party package들을 설치해야 합니다. 바깥에 있는 system의 site-package에다 하면 안됨!